<a href="https://colab.research.google.com/github/crossmodalritual/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os

# Configurações de estilo e cores para os gráficos
DEFAULT_PALETTE = "viridis"
FIGSIZE = (12, 8)
sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
})

def mount_google_drive():
    """Monta o Google Drive no ambiente Colab."""
    drive.mount('/content/drive')

def ensure_directory_exists_on_drive(relative_path):
    """Garante que um diretório exista no Google Drive, criando-o se necessário."""
    drive_path = os.path.join('/content/drive/MyDrive', relative_path)
    os.makedirs(drive_path, exist_ok=True)
    return drive_path

def save_fig(fig, filename, drive_folder_path):
    """Salva uma figura no Google Drive com alta resolução."""
    filepath = os.path.join(drive_folder_path, filename)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close(fig)

def save_summary(summary_text, drive_folder_path):
    """Salva um resumo textual no Google Drive."""
    filepath = os.path.join(drive_folder_path, "sumario.txt")
    with open(filepath, "w") as f:
        f.write(summary_text)

def plot_violin(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Cria e salva um gráfico de violino."""
    fig, ax = plt.subplots(figsize=FIGSIZE)
    sns.violinplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE, split=(hue is not None and df[hue].nunique() > 1))
    ax.set_title(title, color='cyan')
    ax.set_xlabel(x_col.replace('_', ' '), color='white')
    ax.set_ylabel(y_col.replace('_', ' '), color='white')
    if hue:
        ax.legend(title=hue.replace('_', ' '), facecolor='black', edgecolor='white', labelcolor='white')
    save_fig(fig, filename, drive_folder_path)

def plot_kde(df, col, title, filename, drive_folder_path, hue=None):
    """Cria e salva um gráfico de densidade (KDE)."""
    fig, ax = plt.subplots(figsize=FIGSIZE)
    if hue:
        for level in df[hue].unique():
            sns.kdeplot(df.loc[df[hue] == level, col], label=level, fill=True)
        ax.legend(title=hue.replace('_', ' '), facecolor='black', edgecolor='white', labelcolor='white')
    else:
        sns.kdeplot(df[col], fill=True)
    ax.set_title(title, color='cyan')
    ax.set_xlabel(col.replace('_', ' '), color='white')
    ax.set_ylabel('Densidade', color='white')
    save_fig(fig, filename, drive_folder_path)

def plot_line(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Cria e salva um gráfico de linha."""
    fig, ax = plt.subplots(figsize=FIGSIZE)
    sns.lineplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE)
    ax.set_title(title, color='cyan')
    ax.set_xlabel(x_col.replace('_', ' '), color='white')
    ax.set_ylabel(y_col.replace('_', ' '), color='white')
    if hue:
        ax.legend(title=hue.replace('_', ' '), facecolor='black', edgecolor='white', labelcolor='white')
    save_fig(fig, filename, drive_folder_path)

def plot_stacked_bar(df, x_col, y_cols, title, filename, drive_folder_path):
    """Cria e salva um gráfico de barras empilhadas."""
    fig, ax = plt.subplots(figsize=FIGSIZE)
    # Garante que 'y_cols' seja uma lista, mesmo que contenha apenas um elemento
    if isinstance(y_cols, str):
        y_cols = [y_cols]
    # Usa 'x_col' como índice para o gráfico de barras empilhadas
    df = df.set_index(x_col)
    df[y_cols].plot(kind='bar', stacked=True, ax=ax, color=['cyan', 'magenta', 'yellow', 'blue', 'green'][:len(y_cols)])
    ax.set_title(title, color='cyan')
    ax.set_xlabel(x_col.replace('_', ' '), color='white')
    ax.set_ylabel('Valores', color='white')
    # Rotaciona os rótulos do eixo x para melhor legibilidade
    plt.xticks(rotation=45, ha='right')
    save_fig(fig, filename, drive_folder_path)


def analise_exploratoria(dados, drive_folder_path):
    """Realiza análise exploratória inicial dos dados."""
    descritivas = dados.describe(include='all')
    save_summary(descritivas.to_string(), drive_folder_path)
    plot_kde(dados, 'desempenho', 'Distribuição do Desempenho', 'desempenho_distribuicao.png', drive_folder_path)

def analisar_impacto_grupo(dados, drive_folder_path):
    """Analisa o impacto do grupo no desempenho."""
    plot_violin(dados, 'grupo', 'desempenho', 'Desempenho por Grupo (Violin)', 'desempenho_grupo_violin.png', drive_folder_path, hue='grupo')
    plot_line(dados, 'satisfacao', 'desempenho', 'Desempenho ao Longo da Satisfação', 'desempenho_satisfacao.png', drive_folder_path, hue='grupo')

def analisar_participacao_forum(dados, drive_folder_path):
    """Analisa a relação entre participação no fórum e desempenho."""
    plot_line(dados, 'participacao_forum', 'desempenho', 'Desempenho ao Longo da Participação no Fórum', 'desempenho_participacao_forum.png', drive_folder_path, hue='grupo')

def analisar_interacoes_colegas(dados, drive_folder_path):
    """Analisa a relação entre interações com colegas e desempenho."""
    plot_line(dados, 'interacoes_colegas', 'desempenho', 'Desempenho ao Longo das Interações com Colegas', 'desempenho_interacoes_colegas.png', drive_folder_path, hue='grupo')

def analisar_centralidade_grau(dados, drive_folder_path):
    """Analisa a relação entre centralidade de grau e desempenho."""
    plot_line(dados, 'centralidade_grau', 'desempenho', 'Desempenho ao Longo da Centralidade de Grau', 'desempenho_centralidade_grau.png', drive_folder_path, hue='grupo')

def analisar_stacked_bar(dados, drive_folder_path):
    """Analisa médias de variáveis por grupo usando gráfico de barras empilhadas."""
    stacked_data = dados.groupby('grupo')[['satisfacao', 'pertencimento', 'participacao_forum']].mean().reset_index()
    plot_stacked_bar(stacked_data, 'grupo', ['satisfacao', 'pertencimento', 'participacao_forum'],
                     'Médias Empilhadas por Grupo', 'medias_empilhadas_grupo.png', drive_folder_path)


if __name__ == "__main__":
    mount_google_drive()
    graficos_drive_path = ensure_directory_exists_on_drive('data')
    data_url = "https://docs.google.com/spreadsheets/d/1qI7rBBqG3HKCqo2lYfXUIPdmnBiKUgsVXEWLTuH-bsI/export?format=csv"
    dados = pd.read_csv(data_url)
    # Limpeza e padronização dos nomes das colunas
    dados.columns = [col.strip().lower().replace(" ", "_") for col in dados.columns]
    # Conversão de tipos de dados, se necessário
    dados['aluno_id'] = dados['aluno_id'].astype('category')

    # Execução das análises
    analise_exploratoria(dados, graficos_drive_path)
    analisar_impacto_grupo(dados, graficos_drive_path)
    analisar_participacao_forum(dados, graficos_drive_path)
    analisar_interacoes_colegas(dados, graficos_drive_path)
    analisar_centralidade_grau(dados, graficos_drive_path)
    analisar_stacked_bar(dados, graficos_drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-7-d90236567135>:56: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(title=hue.replace('_', ' '), facecolor='black', edgecolor='white', labelcolor='white')
